# Interval Around the Current Price
This strategy will provide liquidity around the current price in a fixed interval. We will rebalance our position every update interval as discussed in the beginning of chapter 4. Let pc be the current price and a ∈ [10, 40000], then we will set our position to [pc − a, pc + a] at every update interval.
We will fill the interval as much as possible. However, it is usually not possible to use up both tokens, so the remaining token will be left outside the pool.

Import code dependencies

In [14]:
from datetime import timedelta, date
from typing import Dict
from _decimal import Decimal
import pandas as pd

from demeter import TokenInfo, Actuator, ChainType, MarketInfo, Strategy, EvaluatorEnum, PeriodTrigger, RowData
from demeter.uniswap import UniV3Pool, UniLpMarket

Set pandas output format

In [15]:
pd.options.display.max_columns = None
pd.set_option("display.width", 5000)

Custom Intervals Around the Current Price strategy to add liquidity at constant interval around current price.

In [16]:
class IntervalsAroundtheCurrentPriceStrategy(Strategy):
    def __init__(self, a=10, b=1, update_interval=timedelta(days=1)):
        super().__init__()
        self.a = a
        self.b = b

    def initialize(self):
        lp_market: UniLpMarket = self.broker.markets[market_key]
        current_price = lp_market.market_status.data.price

        lp_market.add_liquidity(current_price - self.a, current_price + self.a)
        self.triggers.append(PeriodTrigger(time_delta=timedelta(days=1), do=self.work))

    def work(self, row_data: RowData):
        lp_market: UniLpMarket = self.broker.markets[market_key]
        current_price = row_data.prices[eth.name]
        if len(lp_market.positions) > 0:
            lp_market.remove_all_liquidity()
            lp_market.even_rebalance(row_data.prices[eth.name])

        lp_market.add_liquidity(current_price - self.a, current_price + self.a)

Main logic to run Actuator, init two token and market with key "market1"

In [17]:
usdc = TokenInfo(name="usdc", decimal=6)  # declare  token0
eth = TokenInfo(name="eth", decimal=18)  # declare token1
pool = UniV3Pool(usdc, eth, 0.05, usdc)  # declare pool
market_key = MarketInfo("market1")

actuator = Actuator()  # declare actuator
broker = actuator.broker
market = UniLpMarket(market_key, pool)

broker.add_market(market)
broker.set_balance(usdc, 1800)
broker.set_balance(eth, 1)

actuator.strategy = IntervalsAroundtheCurrentPriceStrategy(400, 200)

market.data_path = "../data"
market.load_data(ChainType.polygon.name, "0x45dda9cb7c25131df268515131f647d726f50608", date(2023, 8, 13), date(2023, 8, 17))
actuator.set_price(market.get_price_from_data())
# actuator.run()

2023-11-07 14:42:30,106 - INFO - start load files from 2023-08-13 to 2023-08-17...
2023-11-07 14:42:30,238 - INFO - load file complete, preparing...
2023-11-07 14:42:30,950 - INFO - data has been prepared


Run actuator with evaluators and save result to files

In [18]:
actuator.run(evaluator=[
        EvaluatorEnum.max_draw_down,
        EvaluatorEnum.annualized_returns,
        EvaluatorEnum.net_value,
        EvaluatorEnum.profit,
        EvaluatorEnum.net_value_up_down_rate,
        EvaluatorEnum.eth_up_down_rate,
        EvaluatorEnum.position_fee_profit,
        EvaluatorEnum.position_fee_annualized_returns,
        EvaluatorEnum.position_market_time_rate,
    ])
evaluating_result: Dict[EvaluatorEnum, Decimal] = actuator.evaluating_indicator

actuator.save_result(
    path="./result",  # save path
    account=True,  # save account status list as a csv file
    actions=True,  # save actions as a json file and a pickle file
)

2023-11-07 14:42:30,985 - INFO - init strategy...
2023-11-07 14:42:30,990 - INFO - start main loop...
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7200/7200 [00:09<00:00, 730.17it/s]
2023-11-07 14:42:40,857 - INFO - main loop finished
2023-11-07 14:42:40,943 - INFO - Start calculate evaluating indicator...
2023-11-07 14:42:41,022 - INFO - Evaluating indicator has finished it's job.
2023-11-07 14:42:41,023 - INFO - Print actuator summary
2023-11-07 14:42:41,044 - INFO - Backtesting finished, execute time 10.059340000152588s
2023-11-07 14:42:41,128 - INFO - files have saved to ./result/backtest-20231107-144241.account.csv,./result/backtest-20231107-144241.action.json,./result/backtest-20231107-144241.action.pkl


Final account status                              
Token balance in broker       
USDC      :0                        ETH       :0.1900620350322878923504366033
Position value in markets     
market1(UniLpMarket)
token0    :USDC                     token1    :ETH                      fee       :0.0500                   is 0 base :True                     
positions 
   lower_tick  upper_tick                       pending0                          pending1        liquidity
0      199327      203830  2.884224814133616078486391104  0.001859187691259130963917307917  360889513595323

Account balance history                           
                                         net_value USDC                             ETH              market1_net_value       market1_base_uncollected         market1_quote_uncollected       market1_base_in_position       market1_quote_in_position  market1_position_count
2023-08-13 00:00:00  3648.124528036339326191149432    0  0.2080628764642717689010486092  3263

['./result/backtest-20231107-144241.account.csv',
 './result/backtest-20231107-144241.action.json',
 './result/backtest-20231107-144241.action.pkl']